# Command Line Examples

All lines starting `!` are commands that you can execute yourself on the command line

In [1]:
!pwd

/Users/cjm/repos/ontology-access-kit/notebooks


In [2]:
!runoak --help

Usage: runoak [OPTIONS] COMMAND [ARGS]...

  Run the oaklib Command Line.

  A subcommand must be passed - for example: ancestors, terms, ...

  Most commands require an input ontology to be specified:

      runoak -i <INPUT SPECIFICATION> SUBCOMMAND <SUBCOMMAND OPTIONS AND
      ARGUMENTS>

  Get help on any command, e.g:

      runoak viz -h

Options:
  -v, --verbose
  -q, --quiet TEXT
  --save-to TEXT              For commands that mutate the ontology, this
                              specifies where changes are saved to
  --autosave / --no-autosave  For commands that mutate the ontology, this
                              determines if these are automatically saved in
                              place
  -i, --input TEXT            input implementation specification. This is
                              either a path to a file, or an ontology selector
  -I, --input-type TEXT       Input type.
  -a, --add TEXT              additional implementation specification.
  --help      

## Ubergraph Examples

Ubergraph is suitable for both graphy type operations as well as some inference/OWLy operations

The default ubergraph endpoint is used, but you could also set up your own

### Graph operations

OAK provides a simple relation-graph style abstraction over ontologies. This incorporates not only is-a relations,
but others like part-of and develops-from too

Background:

- [Part 4](https://incatools.github.io/ontology-access-kit/intro/tutorial04.html) of the tutorial
- [Obo Graph Interface](https://incatools.github.io/ontology-access-kit/interfaces/obograph.html)

#### Finding all ancestors of a set of terms

The [ancestors](https://incatools.github.io/ontology-access-kit/cli.html#runoak-ancestors) command will find all ancestors of a term, or of a set of terms. It can be parameterized by which relationship types to traverse.


In [4]:
!runoak ancestors --help

Usage: runoak ancestors [OPTIONS] [TERMS]...

  List all ancestors

  Example:

      runoak -i cl.owl ancestors CL:4023094

  Note that ancestors is by default over ALL relationship types

  Constrained to is-a and part-of:

      runoak -i cl.owl ancestors CL:4023094 -p i,BFO:0000050

  Same, on ubergraph:

      runoak -i ubergraph: ancestors CL:4023094 -p i,BFO:0000050

  Search terms can also be used:

      runoak -i cl.owl ancestors 'goblet cell'

  Multiple terms can be passed:

      runoak -i sqlite:go.db ancestors GO:0005773 GO:0005737 -p i,p

  More background:

      https://incatools.github.io/ontology-access-kit/interfaces/obograph.html

Options:
  -p, --predicates TEXT           A comma-separated list of predicates
  -O, --output-type TEXT          Desired output type
  --statistics / --no-statistics  For each ancestor, show statistics.
                                  [default: no-statistics]
  -o, --output FILENAME           Output file, e.g. obo file
  --help       

In [5]:
# all is-a (i.e. subClassOf between named classes) ancestors of a seed set.
# note you can use either a list of IDs or a list of labels (e.g. "neuron"),
# we use IDs/CURIEs here for precision
!runoak  -i ubergraph: ancestors GO:0001750 CL:0000540 -p i

BFO:0000002 ! continuant
BFO:0000004 ! independent continuant
BFO:0000040 ! material entity
CARO:0000000 ! anatomical entity
CARO:0030000 ! biological entity
CL:0000000 ! cell
CL:0000003 ! native cell
CL:0000211 ! electrically active cell
CL:0000255 ! eukaryotic cell
CL:0000393 ! electrically responsive cell
CL:0000404 ! electrically signaling cell
CL:0000540 ! neuron
CL:0000548 ! animal cell
CL:0002319 ! neural cell
CL:0002371 ! somatic cell
COB:0000006 ! material entity
COB:0000017 ! cell
COB:0000018 ! native cell
UBERON:0001062 ! anatomical entity
GO:0005575 ! cellular_component
GO:0001750 ! photoreceptor outer segment
GO:0110165 ! cellular anatomical entity
owl:Thing ! None
CARO:0000013 ! None


In [6]:
# visualize starting from a seed set of terms
!runoak  -i ubergraph: viz --no-view GO:0001750 CL:0000540 -p i,BFO:0000050 -o output/ug-test1.png -O png

#### Generating Graph Diagrams

The [viz](https://incatools.github.io/ontology-access-kit/cli.html#runoak-viz) command will generate a graphviz
diagram seeded from a set of input terms


In [7]:
!runoak viz --help

Usage: runoak viz [OPTIONS] [TERMS]...

  Visualizing an ancestor graph using obographviz

  For general background on what is meant by a graph in OAK, see
  https://incatools.github.io/ontology-access-kit/interfaces/obograph.html

  :: note:    This requires that `obographviz
  <https://github.com/INCATools/obographviz>`_ is installed.

  Example:

      runoak -i sqlite:cl.db viz CL:4023094

  Same query on ubergraph:

      runoak -i ubergraph: viz CL:4023094

  Example, showing only is-a:

      runoak -i sqlite:cl.db viz CL:4023094 -p i

  Example, showing only is-a and part-of, to include Uberon:

      runoak -i sqlite:cl.db viz CL:4023094 -p i,p

  As above, including develops-from:

      runoak -i sqlite:cl.db viz CL:4023094 -p i,p,RO:0002202

  With abbreviation:

      runoak -i sqlite:cl.db viz CL:4023094 -p i,p,d

  We can also limit the number of "hops" from the seed terms; for example, all
  is-a and develops-from ancestors of T-cell, limiting to a distance of 2:

     

![img](output/ug-test1.png)

### Gap Filling

The `--gap-fill` option in the viz command can be used to fill in relationship types between an input set of terms.
If the seed set contains terms that are not directly connected, but a relationship type can be inferred, this
inferred relationship type is used to "gap-fill"

In [9]:
!runoak -i ubergraph: viz --gap-fill GO:0001750 CL:0000540 CL:0000000 NCBITaxon:9606 NCBITaxon:1 -p i,p -o output/ug-gap-fill1.png -O png

![img](output/ug-gap-fill1.png)

### Extracting triples

The `extract-triples` command will perform a simplified version of robot extract



In [10]:
!runoak extract-triples --help

Usage: runoak extract-triples [OPTIONS] [TERMS]...

  Extracts a subontology as triples

  Currently the only endpoint to implement this is ubergraph. Ontobee seems to
  have performance issues with the query

  This will soon be supported in the SqlDatabase/Sqlite endpoint

  Example:

      runoak -v -i ubergraph: extract-triples GO:0005635 CL:0000099 -o
      test.ttl -O ttl

Options:
  -p, --predicates TEXT   A comma-separated list of predicates
  -o, --output FILENAME   Output file, e.g. obo file
  -O, --output-type TEXT  Desired output type
  --help                  Show this message and exit.


In [11]:
!runoak  -i ubergraph:cl extract-triples CL:0000540 | grep -A10 owl:Class | head -50

ns1:CARO_0010004 a owl:Class ;
    ns3:normalizedInformationContent 2.9608045960631606 ;
    ns3:referenceCount 2212401 ;
    rdfs:isDefinedBy ns1:caro.owl ;
    rdfs:subClassOf ns1:CARO_0001010,
        ns1:COB_0000022,
        ns1:NCBITaxon_1,
        ns1:OBI_0100026 ;
    owl:equivalentClass ns1:COB_0000118 .

ns1:CL_0000255 a owl:Class ;
    rdfs:label "eukaryotic cell"^^xsd:string ;
    ns3:normalizedInformationContent 38.061828746974257 ;
    ns3:referenceCount 11215 ;
    ns2:hasDbXref "MESH:D005057"^^xsd:string ;
    rdfs:isDefinedBy ns1:cl.owl ;
    rdfs:subClassOf ns1:CARO_0000013,
        ns1:CL_0000003,
        ns1:COB_0000018 ;
    owl:disjointWith ns1:CL_0000520 ;
    owl:equivalentClass [ a owl:Class ;
            owl:intersectionOf [ ] ],
        [ a owl:Class ;
            owl:intersectionOf [ ] ] .

ns1:CL_0000393 a owl:Class ;
    rdfs:label "electrically responsive cell"^^xsd:string ;
    ns1:IAO_0000115 "A cell whose function is determined by its response to an ele

In [12]:
!runoak  -i ubergraph: extract-triples CL:0000540 | grep -A10 owl:Class | head -50

ns2:CARO_0010004 a owl:Class ;
    ns3:normalizedInformationContent 2.9608045960631606 ;
    ns3:referenceCount 2212401 ;
    rdfs:isDefinedBy ns2:caro.owl ;
    rdfs:subClassOf ns2:CARO_0001010,
        ns2:COB_0000022,
        ns2:NCBITaxon_1,
        ns2:OBI_0100026 ;
    owl:equivalentClass ns2:COB_0000118 .

ns2:CL_0000255 a owl:Class ;
    rdfs:label "eukaryotic cell"^^xsd:string ;
    ns3:normalizedInformationContent 38.061828746974257 ;
    ns3:referenceCount 11215 ;
    ns1:hasDbXref "MESH:D005057"^^xsd:string ;
    rdfs:isDefinedBy ns2:cl.owl ;
    rdfs:subClassOf ns2:CARO_0000013,
        ns2:CL_0000003,
        ns2:COB_0000018 ;
    owl:equivalentClass [ a owl:Class ;
            owl:intersectionOf [ ] ] .

ns2:CL_0000393 a owl:Class ;
    rdfs:label "electrically responsive cell"^^xsd:string ;
    ns2:IAO_0000115 "A cell whose function is determined by its response to an electric signal."^^xsd:string ;
    ns3:normalizedInformationContent 40.268362795149379 ;
    ns3:refer

### Search

The [search](https://incatools.github.io/ontology-access-kit/cli.html#runoak-search) command will search the ontology or ontologies using a search term.


In [13]:
!runoak search --help

Usage: runoak search [OPTIONS] [TERMS]...

  Searches ontology for entities that have a label, alias, or other property
  matching a search term.

  Example:

      runoak -i uberon.obo search limb

  This uses the Pronto implementation to load uberon from disk, and does a
  basic substring search over the labels and synonyms - results are not ranked

  Bioportal (all ontologies):

      runoak -i bioportal: search limb

  (You need to set your API key first)

  This uses the Bioportal API to search over a broad set of ontologies,
  returning a ranked list ranked by relevance. There may be many results, the
  results are streamed, do ctrl^C to stop

  Ubergraph (all ontologies):

      runoak -i ubergraph: search limb

  Ubergraph (one ontology):

      runoak -i ubergraph:uberon search limb

  For more on search, see https://incatools.github.io/ontology-access-
  kit/interfaces/search.html

Options:
  -O, --output-type [obo|obojson|ofn|rdf|json|yaml|csv]
                              

#### Relevancy-ranked search

By default, ubergraph will perform relevancy-ranked search. The implementation of this is endpoint-dependent

In [16]:
!runoak -i ubergraph: search "tentacle" | head -20

NCBITaxon:132411 ! Halcampa duodecimcirrata
NCBITaxon:192593 ! Erpeton tentaculatum
NCBITaxon:2057060 ! Oxyurichthys auchenolepis
UBERON:0012473 ! oral cirrus
UBERON:0013206 ! nasal tentacle


#### Restricting queries to a specific ontology

If you connect to ubergraph using the selector `ubergraph` then the endpoint is treated as a kind of uber-ontology.

Can you use a selector of the form `ubergraph:ONTID` to restrict to a particular ontology within ubergraph:

In [17]:
!runoak -i ubergraph:uberon search "tentacle"

UBERON:0012473 ! oral cirrus
UBERON:0013206 ! nasal tentacle


#### Listing all ontologies



In [18]:
!runoak ontologies --help

Usage: runoak ontologies [OPTIONS]

  Shows ontologies

Options:
  -o, --output FILENAME  Output file, e.g. obo file
  --help                 Show this message and exit.


In [19]:
!runoak -i ubergraph: ontologies

bspo.owl
chebi.owl
cl/cl-base.owl
cob/cob-base.owl
cob/components/cob-to-external.owl
dpo/dpo-base.owl
eco/eco-base.owl
ecto/ecto-base.owl
emapa.owl
envo/envo-base.owl
fbbt/fbbt-base.owl
fbcv/fbcv-base.owl
fbdv/fbdv-base.owl
fypo/fypo-base.owl
go/extensions/go-bfo-bridge.owl
go/go-base.owl
hp/hp-base.owl
ma.owl
maxo/maxo-base.owl
mi.owl
mmo.owl
mondo/mondo-base.owl
ncbitaxon.owl
ncit.owl
mp/mp-base.owl
obi/obi-base.owl
pato/pato-base.owl
pcl/pcl-base.owl
ro/ro-base.owl
so.owl
uberon/bridge/cl-bridge-to-caro.owl
uberon/bridge/cl-bridge-to-fbbt.owl
uberon/bridge/uberon-bridge-to-caro.owl
uberon/bridge/uberon-bridge-to-fbbt.owl
uberon/uberon-base.owl
wbbt/wbbt-base.owl
wbls/wbls-base.owl
wbphenotype/wbphenotype-base.owl
zfa/zfa-base.owl
pr.owl
po.owl
https://raw.githubusercontent.com/PlantPhenoOntology/ppo/master/ppo.owl
apo.owl
mmusdv.owl
foodon.owl
http://translator.renci.org/ubergraph-axioms.ofn


## BioPortal

This implementation is ideal for use cases involving

 - search over diverse multiple ontologies
 - extracting text mappings (in SSSOM) across broad ontologies
 - text annotation
 


#### BioPortal Search

In [20]:
!runoak -i bioportal: search "tentacle" | head -20

BTO:0001357 ! tentacle
http://purl.jp/bio/4/id/200906071014668510 ! tentacle
CEPH:0000256 ! tentacle
http://www.projecthalo.com/aura#Tentacle ! Tentacle
CEPH:0000256 ! tentacle
CTENO:0000018 ! tentacle
BSA:0000109 ! tentacle
CLYH:1000005 ! tentacle
UBERON:0013206 ! nasal tentacle
UBERON:0013206 ! nasal tentacle
CEPH:0000259 ! tentacle pocket
CEPH:0000261 ! tentacle thickness
CEPH:0000257 ! tentacle absence
CEPH:0000258 ! tentacle pad
CTENO:0000121 ! tentacle sheath
CTENO:0000120 ! tentacle epithelium
CTENO:0000152 ! tentacle apparatus
CTENO:0000143 ! tentacle bulb
AMPHX:1000339 ! velar tentacle
CLYH:1000004 ! tentacle system


#### Extracting mappings from BioPortal

BioPortal includes all by all LOOM mappings for all sources

We can extract these using the mappings command

In [26]:
!runoak mappings --help

Usage: runoak mappings [OPTIONS] [TERMS]...

  List all SSSOM mappings in the ontology

  Example (YAML):

      runoak -i db/envo.db mappings

  TODO: TSV

Options:
  -o, --output FILENAME   Output file, e.g. obo file
  -O, --output-type TEXT  Desired output type
  --maps-to-source TEXT
  --help                  Show this message and exit.


In [22]:
!runoak -i bioportal: mappings CTENO:0000018 -O sssom

# curie_map: {}
# license: UNSPECIFIED
# mapping_set_id: temp
subject_id	predicate_id	object_id	mapping_justification	mapping_provider	mapping_tool
http://purl.obolibrary.org/obo/CTENO_0000018	skos:closeMatch	http://purl.obolibrary.org/obo/BTO_0001357	semapv:UnspecifiedMatching	http://data.bioontology.org/metadata/Mapping	LOOM
http://purl.obolibrary.org/obo/CTENO_0000018	skos:closeMatch	http://purl.obolibrary.org/obo/BTO_0001357	semapv:UnspecifiedMatching	http://data.bioontology.org/metadata/Mapping	LOOM
http://purl.obolibrary.org/obo/CTENO_0000018	skos:closeMatch	http://purl.obolibrary.org/obo/BTO_0001357	semapv:UnspecifiedMatching	http://data.bioontology.org/metadata/Mapping	LOOM
http://purl.obolibrary.org/obo/CTENO_0000018	skos:closeMatch	http://purl.obolibrary.org/obo/BTO_0001357	semapv:UnspecifiedMatching	http://data.bioontology.org/metadata/Mapping	LOOM
http://purl.obolibrary.org/obo/CTENO_0000018	skos:closeMatch	http://purl.obolibrary.org/obo/CLYH_1000005	semapv:UnspecifiedMatch

In [23]:
## Write output to a file; after this we will parse with pandas
!runoak -i bioportal: mappings CTENO:0000018 -O sssom -o output/tentacle-bioportal.sssom.tsv

In [24]:
# pandas is a python library that is useful for working with TSV files, including SSSOM
import pandas as pd

In [25]:
pd.read_csv('output/tentacle-bioportal.sssom.tsv', sep='\t', comment='#')

,subject_id,predicate_id,object_id,mapping_justification,mapping_provider,mapping_tool
0,http://purl.obolibrary.org/obo/CTENO_0000018,skos:closeMatch,http://purl.obolibrary.org/obo/BTO_0001357,semapv:UnspecifiedMatching,http://data.bioontology.org/metadata/Mapping,LOOM
1,http://purl.obolibrary.org/obo/CTENO_0000018,skos:closeMatch,http://purl.obolibrary.org/obo/BTO_0001357,semapv:UnspecifiedMatching,http://data.bioontology.org/metadata/Mapping,LOOM
2,http://purl.obolibrary.org/obo/CTENO_0000018,skos:closeMatch,http://purl.obolibrary.org/obo/BTO_0001357,semapv:UnspecifiedMatching,http://data.bioontology.org/metadata/Mapping,LOOM
3,http://purl.obolibrary.org/obo/CTENO_0000018,skos:closeMatch,http://purl.obolibrary.org/obo/BTO_0001357,semapv:UnspecifiedMatching,http://data.bioontology.org/metadata/Mapping,LOOM
4,http://purl.obolibrary.org/obo/CTENO_0000018,skos:closeMatch,http://purl.obolibrary.org/obo/CLYH_1000005,semapv:UnspecifiedMatching,http://data.bioontology.org/metadata/Mapping,LOOM
5,http://purl.obolibrary.org/obo/CTENO_0000018,skos:closeMatch,http://purl.jp/bio/4/id/200906071014668510,semapv:UnspecifiedMatching,http://data.bioontology.org/metadata/Mapping,LOOM
6,http://purl.obolibrary.org/obo/CTENO_0000018,skos:closeMatch,http://purl.obolibrary.org/obo/CEPH_0000256,semapv:UnspecifiedMatching,http://data.bioontology.org/metadata/Mapping,LOOM
7,http://purl.obolibrary.org/obo/CTENO_0000018,skos:closeMatch,http://purl.obolibrary.org/obo/CEPH_0000256,semapv:UnspecifiedMatching,http://data.bioontology.org/metadata/Mapping,LOOM
8,http://purl.obolibrary.org/obo/CTENO_0000018,skos:closeMatch,http://purl.obolibrary.org/obo/BSA_0000109,semapv:UnspecifiedMatching,http://data.bioontology.org/metadata/Mapping,LOOM
9,http://purl.obolibrary.org/obo/CTENO_0000018,skos:closeMatch,http://www.projecthalo.com/aura,NaN,NaN,NaN


#### Text Annotation

The [annotate](https://incatools.github.io/ontology-access-kit/cli.html#runoak-annotate) command will 
perform NER/CR using a source

In [27]:
!runoak annotate --help

Usage: runoak annotate [OPTIONS] [WORDS]...

  Annotate a piece of text using a Named Entity Recognition annotation

  Example:     runoak -i bioportal: annotate "enlarged nucleus in T-cells from
  peripheral blood"

  Currently BioPortal is the only implementation. Volunteers sought to
  implement for OLS.

  See the ontorunner framework for plugins for SciSpacy and OGER

  For more on text annotation, see:

   - <https://incatools.github.io/ontology-access-kit/interfaces/text-
   annotator.html>_

Options:
  -W, --matches-whole-text / --no-W, --no-matches-whole-text
                                  if true, then only show matches that span
                                  the entire input text  [default: no-W]
  --text-file FILENAME            Text file to annotate. Each newline
                                  separated entry is a distinct text.
  -o, --output FILENAME           Output file, e.g. obo file
  -O, --output-type TEXT          Desired output type
  --help             

In [28]:
!runoak -i bioportal: annotate "cephalopods may have tentacular clubs on the end of their tentacles"

match_type: SYN
object_id: MESH:D049830
object_label: Cephalopoda
object_source: https://data.bioontology.org/ontologies/BERO
subject_end: 11
subject_label: CEPHALOPODS
subject_start: 1

---
match_type: SYN
object_id: http://id.nlm.nih.gov/mesh/D049830
object_label: Cephalopoda
object_source: https://data.bioontology.org/ontologies/MDM
subject_end: 11
subject_label: CEPHALOPODS
subject_start: 1

---
match_type: PREF
object_id: NIBIOHN:NHNSFG:1010054
object_label: Cephalopods
object_source: https://data.bioontology.org/ontologies/FGNHNS
subject_end: 11
subject_label: CEPHALOPODS
subject_start: 1

---
match_type: SYN
object_id: MESH/D049830
object_label: Cephalopoda
object_source: https://data.bioontology.org/ontologies/MESH
subject_end: 11
subject_label: CEPHALOPODS
subject_start: 1

---
match_type: PREF
object_id: http://bioontology.org/projects/ontologies/birnlex#birnlex_519
object_label: Cephalopods
object_source: https://data.bioontology.org/ontologies/BIRNLEX
subject_end: 11
subjec

#### Restricting to a single ontology in BioPortal

Just like with Ubergraph, by default bioportal is treated as a single uber-ontology, but you can use a more specific
selector to restrict search, e.g to the [cephalopod](https://obofoundry.org/ontology/ceph) ontology

In [29]:
!runoak -i bioportal:ceph annotate "cephalopods may have tentacular clubs on the end of their tentacles"

match_type: SYN
object_id: NCBITaxon:6605
object_label: Cephalopoda
object_source: https://data.bioontology.org/ontologies/CEPH
subject_end: 11
subject_label: CEPHALOPODS
subject_start: 1

---


## Pronto (for local and remote files)

Pronto can be used for local files in obo format, or rdf/xml following an OBO-like profile; also for the same files hosted on http://obofoundry.org

In [31]:
!runoak -i prontolib:fbbt.obo info "wing vein"

FBbt:00004751 ! wing vein


## Sqlite

One of the most efficient ways to query is via a SQL database.

The [semantic-sql](https://github.com/INCATools/semantic-sql) repo provides ready-made SQLite files for all OBO ontologies plus a number of other ontologies.

See also:

- [Part 7](https://incatools.github.io/ontology-access-kit/intro/tutorial07.html) of the tutorial

#### Retrieving ready-made sqlite files

Use the `sqlite:obo:ONTID` selector to use a ready-made sqlite file.

There is an initial delay while this is download, after this it is stashed using pystow (by default in `~/.data`)

In [33]:
!runoak -i sqlite:obo:envo ancestors -p i,p river

BFO:0000001 ! entity
BFO:0000002 ! continuant
BFO:0000004 ! independent continuant
BFO:0000024 ! fiat object
BFO:0000024 ! fiat object part
BFO:0000030 ! object
BFO:0000040 ! material entity
ENVO:00000022 ! river
ENVO:00000023 ! stream
ENVO:00000029 ! watercourse
ENVO:00000063 ! water body
ENVO:01000618 ! lotic water body
ENVO:01000685 ! water mass
ENVO:01000799 ! astronomical body
ENVO:01000804 ! astronomical object
ENVO:01000813 ! astronomical body part
ENVO:01000819 ! hydrosphere
ENVO:01001476 ! body of liquid
ENVO:01001477 ! liquid astronomical body part
ENVO:01001479 ! fluid astronomical body part
ENVO:01001784 ! compound astronomical body part


In [34]:
!runoak -i sqlite:obo:envo descendants -p i,p river | runoak -i input/envo.db info -O obo -

[Term]
id: ENVO:00000022
name: river
def: "A stream which, through permanent or seasonal flow processes, moves from elevated land towards lower elevations through a definite channel and empties either into a sea, lake, or another river or ends on land as bed seepage and evapotranspiration exceed water supply." [<http://orcid.org/0000-0002-4366-3088>, ADL:FTT, https://en.wikipedia.org/wiki/River]
xref: EcoLexicon:river
xref: FTT:1179
xref: FTT:251
xref: SPIRE:River
xref: SWEETRealm:River
xref: TGN:21105
synonym: "rio" EXACT []
synonym: "braided river" NARROW []
is_a: ENVO:00000023 ! stream
relationship: RO:0002150 ENVO:01000652 ! river channel
relationship: RO:0002220 ENVO:00000143 ! river bank


[Term]
id: ENVO:00000059
name: underground river
def: "A river that is under the surface of the Earth." [MA:ma]
xref: TGN:21187
xref: https://en.wikipedia.org/wiki/Underground_river
is_a: ENVO:00000022 ! river
is_a: ENVO:00000060 ! underground stream


[Term]
id: ENVO:00000101
name: delta
def: 

In [36]:
!runoak -i sqlite:obo:cl info "neuron"

CL:0000540 ! neuron


In [37]:
## terms ending with "neuron" (in label or synonym)
!runoak -i sqlite:obo:cl info "t/neuron$"

CL:4023041 ! L5 extratelencephalic projecting glutamatergic cortical neuron
CL:4023121 ! sst chodl GABAergic cortical interneuron
CL:0000100 ! motor neuron
CL:0000119 ! cerebellar Golgi cell
CL:0000121 ! Purkinje cell
CL:0000207 ! olfactory receptor cell
CL:0000526 ! afferent neuron
CL:0000527 ! efferent neuron
CL:0000561 ! amacrine cell
CL:0000598 ! pyramidal neuron
CL:0000617 ! GABAergic neuron
CL:0000847 ! ciliated olfactory receptor neuron
CL:0000848 ! microvillous olfactory receptor neuron
CL:0000849 ! crypt olfactory receptor neuron
CL:0000850 ! serotonergic neuron
CL:0002609 ! obsolete neuron of cerebral cortex
CL:0005007 ! Kolmer-Agduhr neuron
CL:0005023 ! branchiomotor neuron
CL:0005024 ! somatomotor neuron
CL:0005025 ! visceromotor neuron
CL:0008031 ! cortical interneuron
CL:0008037 ! gamma motor neuron
CL:0008038 ! alpha motor neuron
CL:0009000 ! sensory neuron of spinal nerve
CL:0010012 ! cerebral cortex neuron
CL:0011003 ! magnocellular neurosecretory cell
CL:0011005 ! GAB

#### Validation

See [How to validate an ontology](https://incatools.github.io/ontology-access-kit/howtos/validate-an-obo-ontology.html)

We will use PATO as an example, as it is small

In [38]:
!runoak -i sqlite:obo:pato validate -o output/pato.validation.tsv | head

type: http://www.w3.org/ns/shacl#MinCountConstraintComponent
subject: BFO:0000027
severity: ERROR
predicate: rdfs:label
info: Missing slot (label) for BFO:0000027

type: http://www.w3.org/ns/shacl#MinCountConstraintComponent
subject: CARO:0000000
severity: ERROR
predicate: rdfs:label


In [39]:
## the results are in TSV format conforming to the LinkML/SHACL validation datamodel
## we will explore this using Pandas
pd.read_csv('output/pato.validation.tsv', sep='\t')

,type,subject,severity,instantiates,predicate,object,object_str,source,info
0,http://www.w3.org/ns/shacl#MinCountConstraintC...,BFO:0000027,ERROR,None,rdfs:label,None,None,None,Missing slot (label) for BFO:0000027
1,http://www.w3.org/ns/shacl#MinCountConstraintC...,CARO:0000000,ERROR,None,rdfs:label,None,None,None,Missing slot (label) for CARO:0000000
2,http://www.w3.org/ns/shacl#MinCountConstraintC...,NBO:0000073,ERROR,None,rdfs:label,None,None,None,Missing slot (label) for NBO:0000073
3,http://www.w3.org/ns/shacl#MinCountConstraintC...,NCBITaxon:3176,ERROR,None,rdfs:label,None,None,None,Missing slot (label) for NCBITaxon:3176
4,http://www.w3.org/ns/shacl#MinCountConstraintC...,NCBITaxon:3378,ERROR,None,rdfs:label,None,None,None,Missing slot (label) for NCBITaxon:3378
...,...,...,...,...,...,...,...,...,...
197,https://w3id.org/linkml/validation-model/Depre...,CHEBI:16189,WARNING,None,oio:hasAlternativeId,None,None,None,Deprecated slot (has_alternative_id) for CHEBI...
198,https://w3id.org/linkml/validation-model/Depre...,CHEBI:16189,WARNING,None,oio:hasAlternativeId,None,None,None,Deprecated slot (has_alternative_id) for CHEBI...
199,https://w3id.org/linkml/validation-model/Depre...,CHEBI:16189,WARNING,None,oio:hasAlternativeId,None,None,None,Deprecated slot (has_alternative_id) for CHEBI...
200,https://w3id.org/linkml/validation-model/Depre...,CHEBI:16336,WARNING,None,oio:hasAlternativeId,None,None,None,Deprecated slot (has_alternative_id) for CHEBI...


## Boolean Search

We assume you have read [part 1](https://incatools.github.io/ontology-access-kit/intro/tutorial01.html) of the tutorial.



### passing multiple terms

Most OAK commands take one **or more** arguments (recall the difference between *options* and *arguments*)

These are treated as **OR** terms, so for example if we feed multiple terms into `info` we will get info
on the set of all terms

In [12]:
!runoak -i sqlite:obo:uberon info 'skin of head' 'skin of face' 'skin of scalp'

UBERON:0001084 ! skin of head
UBERON:1000021 ! skin of face
UBERON:8300000 ! skin of scalp


For precision, use CURIEs in these queries:

In [13]:
!runoak -i sqlite:obo:uberon info UBERON:0001084 UBERON:1000021 UBERON:8300000

UBERON:0001084 ! skin of head
UBERON:1000021 ! skin of face
UBERON:8300000 ! skin of scalp


Graph operation commands such as `descendants` will find all descendants where the start nodes are the union
of all search terms:

In [14]:
!runoak -i sqlite:obo:uberon descendants -p i 'skin of face' 'skin of scalp'

UBERON:0001457 ! skin of eyelid
UBERON:0008802 ! cheek pouch
UBERON:0008803 ! skin of cheek
UBERON:0010515 ! brille
UBERON:0012304 ! nasal diverticulum
UBERON:0013640 ! internal cheek pouch
UBERON:0013641 ! external cheek pouch
UBERON:0013766 ! epicanthal fold
UBERON:0015476 ! nose skin
UBERON:0016462 ! periorbital skin
UBERON:0016475 ! skin of forehead
UBERON:0016918 ! upper vermilion
UBERON:0018151 ! skin of upper lip
UBERON:0034765 ! glabella skin
UBERON:1000003 ! dewlap
UBERON:1000015 ! skin of snout
UBERON:1000021 ! skin of face
UBERON:8300000 ! skin of scalp


### Graph queries as search terms

We can also pass in graph queries *as search terms*. These are **expanded** to lists of terms that are substituted
into the query.

Query terms are preceded with a `.` and are parameterized using `//`. 

So for example the query term `.desc//p=i` means "find the descendants over is-a":

In [16]:
!runoak -i sqlite:obo:uberon info .desc//p=i 'skin of face'

UBERON:1000021 ! skin of face
UBERON:0008803 ! skin of cheek
UBERON:0008802 ! cheek pouch
UBERON:1000003 ! dewlap
UBERON:1000015 ! skin of snout
UBERON:0016475 ! skin of forehead
UBERON:0016462 ! periorbital skin
UBERON:0001457 ! skin of eyelid
UBERON:0016918 ! upper vermilion
UBERON:0013640 ! internal cheek pouch
UBERON:0013641 ! external cheek pouch
UBERON:0013766 ! epicanthal fold
UBERON:0012304 ! nasal diverticulum
UBERON:0018151 ! skin of upper lip
UBERON:0015476 ! nose skin
UBERON:0034765 ! glabella skin
UBERON:0010515 ! brille


Multiple search terms are treated as ORs; so for example to get the union of all (1) subtypes of skin (2) parts of the face, we can do:

In [17]:
!runoak -i sqlite:obo:uberon info .desc//p=i 'skin' .desc//p=i,p 'face' | head -20

UBERON:0014931 ! periungual skin
UBERON:0009015 ! upper back skin
UBERON:0009014 ! lower back skin
UBERON:0004264 ! lower leg skin
UBERON:0004263 ! upper arm skin
UBERON:0004262 ! upper leg skin
UBERON:1000021 ! skin of face
UBERON:0013626 ! medial metatarsal pad
UBERON:0013627 ! lateral metatarsal pad
UBERON:0013628 ! pollical pad
UBERON:0013629 ! hallical pad
UBERON:0013622 ! manual autopod pad
UBERON:0013623 ! pedal autopod pad
UBERON:0001517 ! skin of elbow
UBERON:0001513 ! skin of pes
UBERON:0004014 ! labium minora
UBERON:0001518 ! skin of wrist
UBERON:0001519 ! skin of manus
UBERON:0001511 ! skin of leg
UBERON:0001512 ! skin of ankle


We can use `.and` to perform intersections:

In [18]:
!runoak -i sqlite:obo:uberon info .desc//p=i 'skin' .and .desc//p=i,p 'face'

UBERON:1000021 ! skin of face
UBERON:0008803 ! skin of cheek
UBERON:0008802 ! cheek pouch
UBERON:1000003 ! dewlap
UBERON:1000015 ! skin of snout
UBERON:0016475 ! skin of forehead
UBERON:0016462 ! periorbital skin
UBERON:0001457 ! skin of eyelid
UBERON:0016918 ! upper vermilion
UBERON:0013640 ! internal cheek pouch
UBERON:0013641 ! external cheek pouch
UBERON:0013766 ! epicanthal fold
UBERON:0012304 ! nasal diverticulum
UBERON:0018151 ! skin of upper lip
UBERON:0015476 ! nose skin
UBERON:0034765 ! glabella skin
UBERON:0010515 ! brille


We can use `.not` to perform set subtractions:

In [20]:
!runoak -i sqlite:obo:uberon info .desc//p=i 'skin of head' .not .desc//p=i 'skin of face'

UBERON:8300000 ! skin of scalp
UBERON:0001458 ! skin of lip
UBERON:0001459 ! skin of external ear
UBERON:0016919 ! lower vermilion
UBERON:0016915 ! vermilion
UBERON:0012305 ! marginal cutaneous pouch of ear
UBERON:0012458 ! antler velvet
UBERON:0018150 ! skin of lower lip
UBERON:0001084 ! skin of head


### Extracting subsets, performing gap-filling

We can show an end to end example here that shows how to to extract custom subsets

First consider the tree formed by fetching all descendants of a term:


In [22]:
!runoak -i sqlite:obo:uberon tree --gap-fill -p i .desc//p=i 'skin of head'  

* [] **UBERON:0001084 ! skin of head**
    * [i] **UBERON:8300000 ! skin of scalp**
    * [i] **UBERON:1000021 ! skin of face**
        * [i] **UBERON:1000015 ! skin of snout**
        * [i] **UBERON:1000003 ! dewlap**
        * [i] **UBERON:0018151 ! skin of upper lip**
            * [i] **UBERON:0016918 ! upper vermilion**
        * [i] **UBERON:0016475 ! skin of forehead**
            * [i] **UBERON:0034765 ! glabella skin**
        * [i] **UBERON:0016462 ! periorbital skin**
        * [i] **UBERON:0015476 ! nose skin**
        * [i] **UBERON:0012304 ! nasal diverticulum**
        * [i] **UBERON:0010515 ! brille**
        * [i] **UBERON:0008803 ! skin of cheek**
            * [i] **UBERON:0008802 ! cheek pouch**
                * [i] **UBERON:0013641 ! external cheek pouch**
                * [i] **UBERON:0013640 ! internal cheek pouch**
        * [i] **UBERON:0001457 ! skin of eyelid**
            * [i] **UBERON:0013766 ! epicanthal fold**
    * [i] **UBERON:0016915 ! vermilion**
 

Note: all terms are highlighted as all are part of the input set.

Note that this hierarchy is relatively deep - what if we decided to remove intermediate nodes, for example:

* [] UBERON:0001084 ! skin of head
    * [i] **UBERON:8300000 ! skin of scalp** <---- *REMOVE THIS ONE*
    * [i] UBERON:1000021 ! skin of face

However, if we remove this we want to make sure "skin of face" is not left an orphan, it should have the most
direct *entailed* edges after removal.

We do this by:

1. Adding *exclusion* criteria to our query
2. Using the `--gap-fill` option to plug any gaps

In [23]:
!runoak -i sqlite:obo:uberon tree --gap-fill -p i .desc//p=i 'skin of head' .not 'skin of face'

* [] **UBERON:0001084 ! skin of head**
    * [i] **UBERON:8300000 ! skin of scalp**
    * [i] **UBERON:1000015 ! skin of snout**
    * [i] **UBERON:1000003 ! dewlap**
    * [i] **UBERON:0016915 ! vermilion**
        * [i] **UBERON:0016919 ! lower vermilion**
        * [i] **UBERON:0016918 ! upper vermilion**
    * [i] **UBERON:0016475 ! skin of forehead**
        * [i] **UBERON:0034765 ! glabella skin**
    * [i] **UBERON:0016462 ! periorbital skin**
    * [i] **UBERON:0015476 ! nose skin**
    * [i] **UBERON:0012458 ! antler velvet**
    * [i] **UBERON:0012304 ! nasal diverticulum**
    * [i] **UBERON:0010515 ! brille**
    * [i] **UBERON:0008803 ! skin of cheek**
        * [i] **UBERON:0008802 ! cheek pouch**
            * [i] **UBERON:0013641 ! external cheek pouch**
            * [i] **UBERON:0013640 ! internal cheek pouch**
    * [i] **UBERON:0001459 ! skin of external ear**
        * [i] **UBERON:0012305 ! marginal cutaneous pouch of ear**
    * [i] **UBERON:0001458 ! skin of lip

We can export this to `.obo` format:

In [1]:
!runoak -i sqlite:obo:uberon viz --gap-fill -p i .desc//p=i 'skin of head' .not 'skin of face' -O obo

format-version: 1.4

[Term]
id: UBERON:0001084
name: skin of head

[Term]
id: UBERON:0001457
name: skin of eyelid
is_a: UBERON:0001084

[Term]
id: UBERON:0001458
name: skin of lip
is_a: UBERON:0001084

[Term]
id: UBERON:0001459
name: skin of external ear
is_a: UBERON:0001084

[Term]
id: UBERON:0008802
name: cheek pouch
is_a: UBERON:0008803

[Term]
id: UBERON:0008803
name: skin of cheek
is_a: UBERON:0001084

[Term]
id: UBERON:0010515
name: brille
is_a: UBERON:0001084

[Term]
id: UBERON:0012304
name: nasal diverticulum
is_a: UBERON:0001084

[Term]
id: UBERON:0012305
name: marginal cutaneous pouch of ear
is_a: UBERON:0001459

[Term]
id: UBERON:0012458
name: antler velvet
is_a: UBERON:0001084

[Term]
id: UBERON:0013640
name: internal cheek pouch
is_a: UBERON:0008802

[Term]
id: UBERON:0013641
name: external cheek pouch
is_a: UBERON:0008802

[Term]
id: UBERON:0013766
name: epicanthal fold
is_a: UBERON:0001457

[Term]
id: UBERON:0015476
name: nose skin
is_a: UBERON:0001084

[Term]
id: UBERON

By default the obo has only minimal metadata. There will be more export options in future.

### Applying Changes

We can use [KGCL](https://incatools.github.io/kgcl/) to describe desired changes to an ontology.

In this example we will use the remote version of the FBbt ontology from the OBO library. Of course,
we will not write our changes (which are intentionally silly) back to the central location, but instead to a local file:

In [3]:
!runoak -i prontolib:fbbt.obo apply \
    "rename FBbt:00004751 from 'wing vein' to 'cheesy pizza'"\
    "rename FBbt:00005095 from 'brain' to 'mushroom body pizza'"\
    -o output/fbbt-local.obo -O obo

In [7]:
!runoak -i output/fbbt-local.obo info "t~pizza" -O csv | cut -f1-3

id	label	definition
FBbt:00004751	cheesy pizza	Wing veins are epidermal sclerotizations above hemolymph lacunae that run between the dorsal and the ventral wing surfaces and carry trachea and nerves. The two-dimensional pattern of wing veins is the result of the cuticular differentiation of stripes of epidermal cells on the wing blade, which can be contrasted with the two-dimensional pattern of sensory organs derived from discrete cells.
FBbt:00005095	mushroom body pizza	Anterodorsal portion of the central nervous system. The neuromeres considered to be part of the brain vary by stage. In the larva, the cerebral (supraesophageal) neuromeres, which form a bilateral pair of lobes, constitute the brain (Urbach and Technau, 2003; Li et al., 2014; Crews, 2019). During metamorphosis, the gnathal ganglion separates from the prothoracic neuromere and fuses with the supraesophageal ganglion, to become the ventral part of the adult brain (Ito et al., 2014; Hartenstein et al., 2018; Kendroud et a